<a href="https://colab.research.google.com/github/HasibAlMuzdadid/Machine-Learning-and-Deep-Learning-Projects/blob/main/writing%20like%20shakespeare%20using%20lstm/writing_like_shakespeare_using_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Writing like Shakespeare using LSTM**


A similar task to character-level text generation (but more complicated) is generating Shakespearean poems. We will use a collection of Shakespearean poems as dataset. Using LSTM cells, we can learn longer-term dependencies that span many characters in the text--e.g., where a character appearing somewhere a sequence can influence what should be a different character, much later in the sequence. 



In [ ]:
from __future__ import print_function
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, Masking
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import get_file
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import random
import sys
import io

In [ ]:
# helper function

def build_data(text, Tx = 40, stride = 3):

    # Create a training set by scanning a window of size Tx over the text corpus with stride 3.
    
    # Arguments:
    # text -> string, corpus of Shakespearian poem
    # Tx -> sequence length, number of time-steps (or characters) in one training example
    # stride -> how much the window shifts itself while scanning
    
    # Returns:
    # X -> list of training examples
    # Y -> list of training labels
  
    X = []
    Y = []

    for i in range(0, len(text) - Tx, stride):
        X.append(text[i: i + Tx])
        Y.append(text[i + Tx])
    
    print(f"number of training examples: {len(X)}")
    
    return X, Y


In [ ]:
def vectorization(X, Y, n_x, char_indices, Tx = 40):

    # Convert X and Y (lists) into arrays to be given to a recurrent neural network.
    
    # Arguments:
    # X  
    # Y  
    # Tx -> integer, sequence length
    
    # Returns:
    # x -> array of shape (m, Tx, len(chars))
    # y -> array of shape (m, len(chars))

    
    m = len(X)
    x = np.zeros((m, Tx, n_x), dtype=np.bool)
    y = np.zeros((m, n_x), dtype=np.bool)
    for i, sentence in enumerate(X):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[Y[i]]] = 1
        
    return x, y 

In [ ]:
def sample(preds, temperature=1.0):
    # to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    out = np.random.choice(range(len(chars)), p = probas.ravel())
    return out

In [ ]:
def on_epoch_end(epoch, logs):
  # Function invoked at end of each epoch. Prints generated text.
  None


print("Loading text data...")
text = io.open("/content/shakespeare.txt", encoding='utf-8').read().lower()
#print('corpus length:', len(text))

Tx = 40
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
#print('number of unique characters in the corpus:', len(chars))

print("Creating training set...")
X, Y = build_data(text, Tx, stride = 3)
print("Vectorizing training set...")
x, y = vectorization(X, Y, n_x = len(chars), char_indices = char_indices) 
print("Loading model...")
model = load_model("/content/model_shakespeare_kiank_350_epoch.h5")

Loading text data...
Creating training set...
number of training examples: 31412
Vectorizing training set...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Loading model...


In [ ]:
def generate_output():
    generated = ''
    #sentence = text[start_index: start_index + Tx]
    #sentence = '0'*Tx
    usr_input = input("Write the beginning of your poem, the Shakespeare machine will complete it. Your input is: ")
    # zero pad the sentence to Tx characters.
    sentence = ('{0:0>' + str(Tx) + '}').format(usr_input).lower()
    generated += usr_input 

    sys.stdout.write("\n\nHere is your poem: \n\n") 
    sys.stdout.write(usr_input)
    for i in range(400):

        x_pred = np.zeros((1, Tx, len(chars)))

        for t, char in enumerate(sentence):
            if char != '0':
                x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature = 1.0)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()

        if next_char == '\n':
            continue

A model has already been trained for ~1000 epochs on a collection of Shakespearean poems. 

Let's train the model for one more epoch. When it finishes training for an epoch, we can run `generate_output`, which will ask for an input (`<`40 characters). The poem will start with the input sentence and our RNN Shakespeare will complete the rest of the poem! For example, trying "Forsooth this maketh no sense" (without the quotation marks!). Depending on whether we include the space at the end, our results might also differ.

In [ ]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y, batch_size=128, epochs=1, callbacks=[print_callback])

246/246 [==============================] - 88s 340ms/step - loss: 2.5371


In [ ]:
generate_output()

Write the beginning of your poem, the Shakespeare machine will complete it. Your input is: Forsooth this maketh no sense


Here is your poem: 

Forsooth this maketh no sensew,
thy praise lend with girful onour womts imferlight,
and faves, and eorhty forse though thou self
but thy him thas thee but this sinch thoug, hit prosse.
seand thou of deleck beauty thy behetse,
lide astes that i to bothed me in dhow stand.
thoe i ormom'ntence to mitafaon enceess.
reciriks that were hadning of a bor leges,
in that the by goed mancee part aiaking,
that thee denerver and one of to